## Step 1: Prepare the Training and Evaluation Dataset

In [1]:
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
(X.shape, y.shape)

((442, 10), (442,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([ 0.03807591,  0.05068012,  0.06169621,  0.02187239, -0.0442235 ,
         -0.03482076, -0.04340085, -0.00259226,  0.01990749, -0.01764613]),
  151.0),
 (array([-0.00188202, -0.04464164, -0.05147406, -0.02632753, -0.00844872,
         -0.01916334,  0.07441156, -0.03949338, -0.06833155, -0.09220405]),
  75.0)]

## Step 2: Launching the Spark Session with Rikai Extension

In [3]:
from example import spark

23/01/06 10:12:24 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
23/01/06 10:12:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/da/.cache/pants/named_caches/pex_root/installed_wheels/8f254bc20b539246427b2913639b8a0258db76ab54ba91fbbebb8dc8c36183c1/pyspark-3.3.1-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/da/.ivy2/cache
The jars for the packages stored in: /home/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-afc81846-0a94-42d6-96fe-a83860fa8466;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.2.0-SNAPSHOT in local-ivy-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 151ms :: artifacts dl 8ms
	:: modules in use:
	ai.eto#rikai_2.12;0.2.0-SNAPSHOT from local-ivy-cache in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from 

23/01/06 10:12:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 10:12:31 WARN SparkContext: The path file:///home/da/.ivy2/jars/ai.eto_rikai_2.12-0.2.0-SNAPSHOT.jar has been added already. Overwriting of added paths is not supported in the current version.


## Step 3: Training and Logging using MLflow

In [4]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.linear_model import LinearRegression


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
model = LinearRegression()
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model.fit(X, y)

    registered_model_name = f"{getpass.getuser()}_sklearn_lr"
    log_model(model, registered_model_name)


/home/da/.cache/pants/named_caches/pex_root/venvs/d6a7829465a74d6f7ee1c45b5eb7d6677ce27680/acc1aadce130340dedc6938d0bdd4c7abb8a0412/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/01/06 10:12:38 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'da_sklearn_lr' already exists. Creating a new version of this model...
2023/01/06 10:12:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_sklearn_lr, version 4
Created version '4' of model 'da_sklearn_lr'.
/tmp/pants-sandbox-jlmjuI/python/rikai/spark/sql/codegen/mlflow_logger.py:144: UserWarning: value of rikai.output.schema is None or empty and will not be populated to MLflow
  warnings.warn(


## Step 4: Create the model using the registered MLflow uri

In [5]:
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


+----------------+------+-----------------------+-------+
|name            |flavor|uri                    |options|
+----------------+------+-----------------------+-------+
|mlflow_sklearn_m|      |mlflow:///da_sklearn_lr|       |
+----------------+------+-----------------------+-------+



## Step 5: predict using the registered Rikai model

In [7]:
from liga.sklearn.numpy import array_to_literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {array_to_literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

AnalysisException: Column 'mlflow_sklearn_m' does not exist. Did you mean one of the following? []; line 3 pos 13;
'Project [unresolvedalias('ML_PREDICT('mlflow_sklearn_m, array(0.038075906433423026, cast(0.05068011873981862 as decimal(18,18)), 0.061696206518683294, cast(0.0218723855140367 as decimal(18,18)), cast(-0.04422349842444599 as decimal(18,18)), cast(-0.03482076283769895 as decimal(18,18)), cast(-0.04340084565202491 as decimal(18,18)), -0.002592261998183278, 0.019907486170462722, cast(-0.01764612515980379 as decimal(18,18)))), None)]
+- OneRowRelation


In [8]:
spark.sql(f"""
select  ML_PREDICT(mlflow_sklearn_m, {array_to_literal(X[1])})
""").toPandas()

AnalysisException: Column 'mlflow_sklearn_m' does not exist. Did you mean one of the following? []; line 2 pos 19;
'Project [unresolvedalias('ML_PREDICT('mlflow_sklearn_m, array(-0.0018820165277906047, cast(-0.044641636506989144 as decimal(19,19)), cast(-0.051474061238800654 as decimal(19,19)), cast(-0.02632752814785296 as decimal(19,19)), cast(-0.008448724111216851 as decimal(19,19)), cast(-0.019163339748222204 as decimal(19,19)), cast(0.07441156407875721 as decimal(19,19)), cast(-0.03949338287409329 as decimal(19,19)), cast(-0.0683315470939731 as decimal(19,19)), cast(-0.092204049626824 as decimal(19,19)))), None)]
+- OneRowRelation
